In [ ]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from bs4 import BeautifulSoup
import pandas as pd
import time
import re

In [5]:
# Read the Excel file "Posibles Clientes.xlsx" into a pandas DataFrame
df = pd.read_excel("C:/Users/juans/Documents/cv/Proyectos/Extraccion de Datos/Armar Leads Locales/Posibles Clientes.xlsx")
data_filter = df[df["Unnamed: 2"].isnull()]
data_filter

,Url,Locales,Unnamed: 2,Unnamed: 3
0,https://www.google.com/maps/place/LOMITOS+MARC...,LOMITOS MARCELO,NaN,NaN
1,https://www.google.com/maps/place/Lomitos+LA+I...,Lomitos LA ISLA,NaN,NaN
2,https://www.google.com/maps/place/Lomitos+348+...,Lomitos 348 Barrio Jardin,NaN,NaN
3,https://www.google.com/maps/place/Muy+mucho+-+...,Muy mucho - Hambre de Lomito,NaN,NaN
4,https://www.google.com/maps/place/Lomitos+348/...,Lomitos 348,NaN,NaN
...,...,...,...,...
533,https://www.google.com/maps/place/Al+Buche/dat...,Al Buche,NaN,NaN
539,https://www.google.com/maps/place/Punto+33+B%C...,Punto 33 B° Las Flores,NaN,https://instagram.com/punto33.cba
551,https://www.google.com/maps/place/McDonald%27s...,McDonald's,NaN,https://www.mcdonalds.com.ar/
552,https://www.google.com/maps/place/McDonald%27s...,McDonald's,NaN,https://www.mcdonalds.com.ar/


In [ ]:
# Regular expression pattern to match phone numbers
tel_pattern = re.compile(r"^/+?/d{0,4}[/s-]?/d{3,4}[/s-]?/d{4}$")

# Function to filter and extract valid phone numbers from a list of elements
def filter_number(number_list: list):
    for i in number_list:
        txt = i.get_text().strip()  # Extract and clean text from the element
        if tel_pattern.match(txt):  # Check if the text matches the phone number pattern
            return txt  # Return the valid phone number
    return None  # Return None if no valid phone number is found

# Function to scrape data from URLs in the DataFrame
def scrapper(df=data_filter):
    for row_index, row_data in df.iterrows():  # Iterate through each row in the DataFrame
        url = row_data["Url"]  # Get the URL from the current row
        try:
            driver.get(url)  # Open the URL in the browser
            time.sleep(1)  # Wait for the page to load
            html = driver.page_source  # Get the page source
            soup = BeautifulSoup(html)  # Parse the page source with BeautifulSoup
            elements = soup.find_all("div", class_="Io6YTe")  # Find all relevant elements
            telef = filter_number(elements)  # Extract the phone number from the elements
            df.at[row_index, "Unnamed: 2"] = telef  # Update the DataFrame with the phone number
            print(f"Fila {row_index}: {telef if telef else 'No encontrado'}")  # Print the result
        except Exception as e:
            # Handle exceptions and log errors
            print(f"Error en fila {row_index}, URL {url}: {str(e)}")
            df.at[row_index, "Unnamed: 2"] = None  # Assign None in case of an error

    return df  # Return the updated DataFrame

# Initialize the Selenium WebDriver
driver = webdriver.Chrome()

# Call the scrapper function to process the data
df_result = scrapper(data_filter)

# Merge the original DataFrame with the scraped results
result = pd.merge(df, df_result, how="right")

# Save the final DataFrame to an Excel file
result.to_excel("C:/Users/juans/Documents/cv/Proyectos/Extraccion de Datos/Armar Leads Locales/Dataset Final/Cba_final.xlsx", index=False)